# DS-SF-34 | 05 | _k_-Nearest Neighbors | Assignment | Starter Code

## _k_-Nearest Neighbors on the Boston Dataset

In [70]:
import os

import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 10)

from sklearn import preprocessing, neighbors, model_selection

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

In [71]:
df = pd.read_csv(os.path.join('..', 'datasets', 'dataset-05-boston.csv'))

In [72]:
df

,CRIM,ZN,INDUS,CHAS,NOX,...,TAX,PTRATIO,BLACK,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,...,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,...,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,...,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,...,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,...,222,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,...,273,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0,0.573,...,273,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0,0.573,...,273,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0,0.573,...,273,21.0,393.45,6.48,22.0


In [25]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
CRIM,506.0,3.613524,8.601545,0.00632,0.082045,0.25651,3.677082,88.9762
ZN,506.0,11.363636,23.322453,0.00000,0.000000,0.00000,12.500000,100.0000
INDUS,506.0,11.136779,6.860353,0.46000,5.190000,9.69000,18.100000,27.7400
CHAS,506.0,0.069170,0.253994,0.00000,0.000000,0.00000,0.000000,1.0000
NOX,506.0,0.554695,0.115878,0.38500,0.449000,0.53800,0.624000,0.8710
...,...,...,...,...,...,...,...,...
TAX,506.0,408.237154,168.537116,187.00000,279.000000,330.00000,666.000000,711.0000
PTRATIO,506.0,18.455534,2.164946,12.60000,17.400000,19.05000,20.200000,22.0000
BLACK,506.0,356.674032,91.294864,0.32000,375.377500,391.44000,396.225000,396.9000
LSTAT,506.0,12.653063,7.141062,1.73000,6.950000,11.36000,16.955000,37.9700


In [38]:
df.MEDV.quantile(.2)

15.3

The Boston dataset concerns itself with housing values in suburbs of Boston.  A description of the dataset is as follows:

- CRIM: per capita crime rate by town
- ZN: proportion of residential land zoned for lots over 25,000 sqft
- INDUS: proportion of non-retail business acres per town
- CHAS: Charles River binary/dummy variable (= 1 if tract bounds river; 0 otherwise)
- NOX: nitric oxides concentration (parts per 10 million)
- RM: average number of rooms per dwelling
- AGE: proportion of owner-occupied units built prior to 1940
- DIS: weighted distances to five Boston employment centers
- RAD: index of accessibility to radial highways
- TAX: full-value property-tax rate (per ten thousands of dollars)
- PTRATIO: pupil-teacher ratio by town
- B: 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
- LSTAT: % lower status of the population
- MEDV: Median value of owner-occupied homes (in thousands of dollars)

> ### Question 1.  Let's first categorize `MEDV` to 4 groups: Bottom 20% as Level 1, next 30% as Level 2, next 30% categorized as Level 3, and the top 20% as Level 4.  Please create a new variable `MEDV_Category` that stores the level number

In [73]:
len(df)

506

In [74]:
# TODO

level_2 = ((df.MEDV < df.MEDV.quantile(0.5)) & (df.MEDV >= df.MEDV.quantile(0.2)))
level_3 = ((df.MEDV < df.MEDV.quantile(0.7)) & (df.MEDV >= df.MEDV.quantile(0.5)))
level_4 = (df.MEDV >= df.MEDV.quantile(0.7))

df['MEDV_Category']='1'
df.loc[df.MEDV == level_2, 'MEDV_Category'] = '2'
df.loc[df.MEDV == level_3, 'MEDV_Category'] = '3'
df.loc[df.MEDV == level_4, 'MEDV_Category'] = '4'

# count of level 1 
print len(df) - level_2.sum() - level_3.sum() - level_4.sum()
# count of level 2 
level_2 *= 1
print level_2.sum()
# count of level 3 
level_3 *= 1
print level_3.sum()
# count of level 4 
level_4 *= 1
print level_4.sum()


101
150
103
152


## Our goal is to predict `MEDV_Category` based on `RM`, `PTRATIO`, and `LSTAT`

> ### Question 2.  First normalize `RM`, `PTRATIO`, and `LSTAT`.  By normalizing, we mean to scale each variable between 0 and 1 with the lowest value as 0 and the highest value as 1

In [75]:
# TODO
df

,CRIM,ZN,INDUS,CHAS,NOX,...,PTRATIO,BLACK,LSTAT,MEDV,MEDV_Category
0,0.00632,18.0,2.31,0,0.538,...,15.3,396.90,4.98,24.0,1
1,0.02731,0.0,7.07,0,0.469,...,17.8,396.90,9.14,21.6,1
2,0.02729,0.0,7.07,0,0.469,...,17.8,392.83,4.03,34.7,1
3,0.03237,0.0,2.18,0,0.458,...,18.7,394.63,2.94,33.4,1
4,0.06905,0.0,2.18,0,0.458,...,18.7,396.90,5.33,36.2,1
...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,...,21.0,391.99,9.67,22.4,1
502,0.04527,0.0,11.93,0,0.573,...,21.0,396.90,9.08,20.6,1
503,0.06076,0.0,11.93,0,0.573,...,21.0,396.90,5.64,23.9,1
504,0.10959,0.0,11.93,0,0.573,...,21.0,393.45,6.48,22.0,1


In [77]:
X = df[['RM', 'PTRATIO', 'LSTAT']]

scaler = preprocessing.MinMaxScaler().fit(X)

X = scaler.transform(X)

In [79]:
c = df.MEDV_Category

> ### Question 3.  Run a _k_-Nearest Neighbor classifier with 5 nearest neighbors and report your misclassification error; set `weights` to `uniform`

In [81]:
# TODO
model = neighbors.KNeighborsClassifier(n_neighbors = 5, weights='uniform').\
    fit(X, c)

Answer: TODO

> ### Question 4.  Is this error reliable?

In [ ]:
    
    model.score(X, c)

> ### Question 5.  Now use 10-fold cross-validation to choose the most efficient `k`

In [ ]:
# TODO

> ### Question 6.  Explain your findings

Answer: TODO

> ### Question 7.  Train your model with the optimal `k` you found above (don't worry if it changes from time to time - if that is the case use the one that is usually the best)

In [ ]:
# TODO

Answer: TODO

> ### Question 8.  After training your model with that `k`, use it to predict the class of a neighborhood with `RM = 2`, `PRATIO = 19`, and `LSTAT = 3.5`

In [ ]:
# TODO

Answer: TODO